# Test benchmark

## Setup

In [ ]:
import sys
import subprocess
from antidote import __version__, is_compiled
print(f"""
== Python ==
{sys.version}

== Antidote =
{__version__} {'(cython)' if is_compiled() else ''}
""")

### Imitating a really big project

Creating 4000 different dependencies, all singletons, in all the different ways that actually impact test utilities.

In [ ]:
from antidote import factory, Service, implementation, Factory

n = 1000

services = [
    type(f"Service{i}", (Service,), {})
    for i in range(n)
]

def make_factory(Output):
    @factory
    def f() -> Output:
        return Output()
    
    return f

factories = dict()
for i in range(n):
    Output = type(f"Ouput{i}", (object,), {})
    factories[Output] = make_factory(Output)

def make_implementation(Interface, Impl):
    @implementation(Interface)
    def f():
        return Impl
    
    return f

def make_factory_method(Output):
    def f(self) -> Output:
        return Output()
    
    return f

factories = dict()
for i in range(n, 2 * n):
    Output = type(f"Ouput{i}", (object,), {})
    factories[Output] = type(f"Factory{i}", (Factory,), {"__call__": make_factory_method(Output)})

    
implementations = dict()
for i in range(n):
    Interface = type(f"Interface{i}", (object,), {})
    Impl = type(f"Impl{i}", (Interface, Service), {})
    implementations[Interface] = make_implementation(Interface, Impl)

# Instantiating all dependencies
from antidote import world

for service in services:
    world.get(service)

for output, factory in factories.items():
    world.get(output @ factory)
    
for interface, impl in implementations.items():
    world.get(interface @ impl)

## Benchmarks

In [ ]:
def bench_empty():
    with world.test.empty():
        pass
    
%timeit bench_empty()

In [ ]:
def bench_new():
    with world.test.new():
        pass
    
%timeit bench_new()

In [ ]:
def bench_clone():
    with world.test.clone():
        pass
    
%timeit bench_clone()

In [ ]:
def bench_clone_keeping_singletons():
    with world.test.clone(keep_singletons=True):
        pass
    
%timeit bench_clone_keeping_singletons()

In [ ]:
x = object()
with world.test.clone():
    %timeit world.test.override.singleton(services[0], x)

In [ ]:
x = object()

def static():
    return x

with world.test.clone():
    %timeit world.test.override.factory(services[0])(static)